In [76]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack
import MeCab
import ipadic

In [77]:
df = pd.read_csv("./miss.csv")
df.head()

,Item.Description,Item.Manufacturer,Item.PartsNo,問題点.No,問題点.内容,問題点.原因,問題点.発覚日時,問題点.Project,対策.応急処置,対策.恒久対策
0,電子レンジ,F社,PN00021,1,電源が入らない,設定の誤り,2025/03/10 10:27,仕事,専門業者に依頼した,二重確認の徹底
1,ノートパソコン,G社,PN00023,2,焦げ付く,システムのバグ,2025/04/09 04:36,仕事,謝罪と訂正を行った,パスワードを再設定
2,モバイルバッテリー,A社,PN00003,3,焦げ付く,ウイルス感染,2025/05/29 18:28,家事,定期的なメンテナンスを導入,専門業者に依頼した
3,テレビ,D社,PN00007,4,誤って操作した,不注意,2024/07/04 15:46,デジタルライフ,専門業者に依頼した,保管場所を決める
4,眼鏡,B社,PN00010,5,誤って操作した,設定の誤り,2025/05/17 07:14,通勤,専門業者に依頼した,パスワードを再設定


In [78]:
df_X = df.drop(
    [
        "問題点.原因",
        "問題点.No",
        "対策.応急処置",
        "対策.恒久対策",
    ],
    axis=1,
)
df_y = df["問題点.原因"]

text_columns = [
    "Item.Description", 
    "問題点.内容",
]
categorical_columns = ["Item.Manufacturer", "Item.PartsNo", "問題点.Project"]

df_X.head()

,Item.Description,Item.Manufacturer,Item.PartsNo,問題点.内容,問題点.発覚日時,問題点.Project
0,電子レンジ,F社,PN00021,電源が入らない,2025/03/10 10:27,仕事
1,ノートパソコン,G社,PN00023,焦げ付く,2025/04/09 04:36,仕事
2,モバイルバッテリー,A社,PN00003,焦げ付く,2025/05/29 18:28,家事
3,テレビ,D社,PN00007,誤って操作した,2024/07/04 15:46,デジタルライフ
4,眼鏡,B社,PN00010,誤って操作した,2025/05/17 07:14,通勤


In [79]:
X_train, X_test, y_train, y_test = train_test_split(
    df_X, df_y, test_size=0.2, random_state=42
)

print(f"学習データ数: {len(X_train)}")
print(f"テストデータ数: {len(X_test)}")

学習データ数: 160
テストデータ数: 40


In [80]:
def tokenize_text(text):
    tagger = MeCab.Tagger(f"-O wakati {ipadic.MECAB_ARGS}")
    return tagger.parse(text).strip()

In [81]:
tfidf_vectorizers = {}
X_tfidf_features = []

for col in text_columns:
    print(f"processing text column: {col}")
    cleaned_text = df_X[col].fillna("").apply(tokenize_text)
    vectorizer = TfidfVectorizer(min_df=1, max_df=0.9, ngram_range=(1, 2))
    X_tfidf_features.append(vectorizer.fit_transform(cleaned_text))
    tfidf_vectorizers[col] = vectorizer

processing text column: Item.Description
processing text column: 問題点.内容


In [82]:
encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=True)
X_categorical_features = encoder.fit_transform(df_X[categorical_columns])

print(f"カテゴリ特徴量の形状: {X_categorical_features.shape}")

カテゴリ特徴量の形状: (200, 44)


In [83]:
all_features = hstack(X_tfidf_features + [X_categorical_features])

print(f"結合された全特徴量の形状: {all_features.shape}")

結合された全特徴量の形状: (200, 162)


In [84]:
X_train_combined, X_test_combined, y_train, y_test = train_test_split(
    all_features,
    df_y,
    test_size=0.2,
    random_state=42,
)

print(f"\n最終的な学習データ数: {X_train_combined.shape[0]}")
print(f"最終的なテストデータ数: {X_test_combined.shape[0]}")


最終的な学習データ数: 160
最終的なテストデータ数: 40


In [85]:
model = SVC(kernel="linear", C=1.0, random_state=42)
model.fit(X_train_combined, y_train)

,C,1.0
,kernel,'linear'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [86]:
y_pred = model.predict(X_test_combined)

print(classification_report(y_test, y_pred, zero_division=0))

              precision    recall  f1-score   support

      うっかりミス       1.00      1.00      1.00         2
     ウイルスの感染       1.00      1.00      1.00         3
     システムのバグ       1.00      0.50      0.67         2
   ネットワークの問題       1.00      1.00      1.00         2
     バッテリー切れ       1.00      1.00      1.00         1
    ヒューマンエラー       1.00      0.50      0.67         2
         不注意       1.00      0.75      0.86         4
   多忙による見落とし       0.80      1.00      0.89         8
        接続不良       1.00      1.00      1.00         2
      物理的な衝撃       1.00      0.50      0.67         2
        確認不足       0.67      1.00      0.80         6
        経年劣化       1.00      1.00      1.00         1
       設定の誤り       1.00      1.00      1.00         2
          過熱       0.00      0.00      0.00         1
     電力供給の問題       1.00      1.00      1.00         2

    accuracy                           0.88        40
   macro avg       0.90      0.82      0.84        40
weighted avg       0.89   

In [87]:
y_pred_dec = model.decision_function(X_test_combined)
X_test_labels = X_test.reset_index().drop(["index"], axis=1)
class_labels = model.classes_
for i in range(len(y_pred_dec)):
    y_sorted = sorted(list(zip(class_labels, y_pred_dec[i])), key=lambda x: x[1], reverse=True)
    y_sorted = list(map(lambda x: (x[0], round(float(x[1]), 3)), y_sorted))
    print(X_test_labels.iloc[i])
    print(y_sorted[:3])
    print(y_test.iloc[i])
    print("------------------------------------------------------")

Item.Description                 名刺入れ
Item.Manufacturer                  D社
Item.PartsNo                  PN00028
問題点.内容                      パスワードを再設定
問題点.発覚日時             2025/03/09 09:12
問題点.Project                        学習
Name: 0, dtype: object
[('多忙による見落とし', 17.315), ('ネットワークの問題', 16.31), ('電力供給の問題', 15.302)]
多忙による見落とし
------------------------------------------------------
Item.Description                  充電器
Item.Manufacturer                  C社
Item.PartsNo                  PN00005
問題点.内容                         接続できない
問題点.発覚日時             2024/11/05 06:18
問題点.Project                   デジタルライフ
Name: 1, dtype: object
[('確認不足', 17.315), ('不注意', 16.307), ('電力供給の問題', 15.279)]
過熱
------------------------------------------------------
Item.Description                 エアコン
Item.Manufacturer                  B社
Item.PartsNo                  PN00016
問題点.内容                          異音がする
問題点.発覚日時             2025/01/01 10:48
問題点.Project                        学習
Name: 2, dtype: object
